In [77]:
from abc import ABC, abstractmethod

In [78]:
class Runnable(ABC):

    @abstractmethod
    def invoke(input_data):
        pass

In [79]:
import random 

class NakliLLM(Runnable):

    def __init__(self):
        print("NakliLLM initialized")

    def invoke(self, prompt):
        response_list = [
            "Delhi is the capital of India.",
            "IPL is a T20 cricket league in India.",
            "Python is a programming language."
        ]

        return {'response': random.choice(response_list)}

    def predict(self, promt):
        response_list = [
            "Delhi is the capital of India.",
            "IPL is a T20 cricket league in India.",
            "Python is a programming language."
        ]

        return {'response': random.choice(response_list + "\npredict method is depricated, use invoke instead.")}

In [80]:
llm = NakliLLM()

NakliLLM initialized


In [81]:
class NakliPromptTemaplate(Runnable):

    def __init__(self, template, input_variables):
        self.template = template
        self.input_variables = input_variables

    def invoke(self, input_dict):
        return self.template.format(**input_dict)

    def format(self, input_dict):
        return self.template.format(**input_dict + "\npredict method is depricated, use invoke instead.")

In [82]:
class NakliStrOutParser(Runnable):

    def __init__(self):
        pass

    def invoke(self, llm_output):
        return llm_output['response']

    def parse(self, llm_output):
        return llm_output['response'] + "\npredict method is depricated, use invoke instead."

In [83]:
class RunnableConnector(Runnable):

    def __init__(self, runnable_list):
        self.runnable_list = runnable_list

    def invoke(self, input_data):

        for runnable in self.runnable_list:
            input_data = runnable.invoke(input_data)

        return input_data

In [84]:
template = NakliPromptTemaplate(
    template="Write a {length} article on {topic}",
    input_variables=["length", "topic"]
)

In [85]:
llm = NakliLLM()

NakliLLM initialized


In [86]:
parser = NakliStrOutParser()

In [87]:
chain = RunnableConnector([template, llm, parser])

In [88]:
chain.invoke({"length": "long", "topic": "India"})

'Delhi is the capital of India.'

In [89]:
template1 = NakliPromptTemaplate(
    template="Write a joke about {topic}",
    input_variables=["topic"]
)

In [90]:
template2 = NakliPromptTemaplate(
    template="Explain the follwoing joke {response}",
    input_variables=["response"]
)

In [91]:
llm = NakliLLM()

NakliLLM initialized


In [93]:
parser = NakliStrOutParser()

In [97]:
chain1 = RunnableConnector([template1, llm])

In [99]:
chain1.invoke({"topic": "AI"})

{'response': 'IPL is a T20 cricket league in India.'}

In [100]:
chain2 = RunnableConnector([template2, llm, parser])

In [101]:
chain2.invoke({"response": "this is a joke about AI"})

'Python is a programming language.'

In [103]:
final_chain = RunnableConnector([chain1, chain2])

In [105]:
final_chain.invoke({"topic": "AI"})

'IPL is a T20 cricket league in India.'